In [3]:
%load_ext autoreload
%autoreload 2

###Importing all the useful librairies

import time
import json
import bz2
import pandas as pd
import numpy as np
import re
from pathlib import Path
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import glob

import warnings; warnings.simplefilter('ignore')
import os, codecs, string, random
import numpy as np
from numpy.random import seed as random_seed
from numpy.random import shuffle as random_shuffle
import matplotlib.pyplot as plt
%matplotlib inline  

seed = 42
random.seed(seed)
np.random.seed(seed)

#NLP libraries
import spacy, nltk, gensim, sklearn
#import pyLDAvis.gensim

#Vader
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#Scikit imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

In [4]:
df = pd.read_csv("c:/data/tot_2020.csv")

In [5]:
df.head(10)

,0,Q5268447,2020-01-17-000357,male,[ The delay ] will have an impact [ on Slough ] but that might be mitigated by the fact we are going to have this Western Rail Link to Heathrow. It looks like that may come in sooner than Crossrail.
0,1,Q4864119,2020-04-02-000239,male,[ The scheme ] treats addiction as an illness ...
1,2,Q816459,2020-03-19-000276,male,[ These ] actions will allow households who ha...
2,3,Q56255401,2020-01-08-000594,male,"11 straight weeks of pre-season,"
3,4,Q26923564,2020-02-21-000455,male,2019 was a landmark year for Fiverr as we comp...
4,5,Q4749380,2020-04-01-000532,female,7pm is when most hospitals change shifts. That...
5,6,Q5106592,2020-01-29-001076,male,"A collaborative opioid stewardship program, ac..."
6,7,Q771586,2020-01-26-000499,male,a few of the candidates who will do better in ...
7,8,Q16195572,2020-02-29-000491,male,a lack of resources and government funding
8,9,Q7322163,2020-03-23-000896,female,A little over a week old. Thankful for God's g...
9,10,Q18151420,2020-01-10-001448,male,a lot of contacts and a lot of knowledge


In [6]:
df.columns = ['id','quids', 'quoteID', 'gender', 'quote']

In [7]:
women = df.drop(df[df['gender'] != 'female'].index) 

In [8]:
women.head()

,id,quids,quoteID,gender,quote
4,5,Q4749380,2020-04-01-000532,female,7pm is when most hospitals change shifts. That...
8,9,Q7322163,2020-03-23-000896,female,A little over a week old. Thankful for God's g...
15,16,Q230744,2020-01-11-002445,female,And help he always did. For someone who prefer...
16,17,Q7493177,2020-01-23-005418,female,And I think we've come to a time where we are ...
19,20,Q43394564,2020-02-13-007205,female,Apple prices generally peak in January. Betwee...


In [16]:
nlp = spacy.load('en_core_web_sm')

In [13]:
all_quotes = women.drop(['id','quids','quoteID','gender'], axis = 1)

In [14]:
all_quotes.head()

,quote
4,7pm is when most hospitals change shifts. That...
8,A little over a week old. Thankful for God's g...
15,And help he always did. For someone who prefer...
16,And I think we've come to a time where we are ...
19,Apple prices generally peak in January. Betwee...


In [15]:
all_quotes.to_csv(r'c:/data/w/women.txt', header=None, index=None, sep=' ', mode='a')

In [16]:
#type(df["quote"])

#np.savetxt(r'c:\data\np.txt', df.to_string())

In [9]:
books = list()
corpus_root = "c:/data/w/"


with codecs.open(os.path.join(corpus_root,'women.txt'),encoding="utf8") as f:
    books.append(f.read())
    f.close()

In [10]:
print(books[0][0:600])

"7pm is when most hospitals change shifts. That's why. We love you heroes."
"A little over a week old. Thankful for God's goodness."
"And help he always did. For someone who preferred to be behind the scenes, he was at the center of absolutely everything."
"And I think we've come to a time where we are prepared if women are not supported, then women are not going to continue to blindly support men who will not step up and take some leadership on this issue,"
"Apple prices generally peak in January. Between December 2019 and January 2020, apple prices rose 22 percent to a weighted average p


In [11]:
book = books[0]

len(book)
#doc = nlp(book)

16834789

In [12]:
from nltk.corpus import PlaintextCorpusReader
?PlaintextCorpusReader
our_books = PlaintextCorpusReader(corpus_root, '.*.txt')
print(our_books.fileids())

['women.txt']


In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lucie\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [35]:
import nltk


def get_chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]


book_id = {f:n for n,f in enumerate(our_books.fileids())} # dictionary of books
chunks = list()
chunk_class = list() # this list contains the original book of the chunk, for evaluation

limit = 500 # how many chunks total
size = 50 # how many sentences per chunk/page

for f in our_books.fileids():
    sentences = our_books.sents(f)
    print(f)
    #print('Number of sentences:',len(sentences))
    print("hello")
    
    # create chunks
    chunks_of_sents = [x for x in get_chunks(sentences,size)] # this is a list of lists of sentences, which are a list of tokens
    chs = list()
    
    # regroup so to have a list of chunks which are strings
    for c in chunks_of_sents:
        grouped_chunk = list()
        for s in c:
            grouped_chunk.extend(s)
        chs.append(" ".join(grouped_chunk))
    #print("Number of chunks:",len(chs),'\n')
    
    # filter to the limit, to have the same number of chunks per book
    chunks.extend(chs[:limit])
    chunk_class.extend([book_id[f] for _ in range(len(chs[:limit]))])

women.txt
hello


In [19]:
STOPWORDS = spacy.lang.en.stop_words.STOP_WORDS

processed_docs = list()
for doc in nlp.pipe(chunks, batch_size=10):

    # Process document using Spacy NLP pipeline.
    ents = doc.ents  # Named entities

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list and keep only words of length 3 or more.
    doc = [token for token in doc if token not in STOPWORDS and len(token) > 2]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])

    processed_docs.append(doc)
docs = processed_docs
del processed_docs

In [20]:
# Add bigrams too
from gensim.models.phrases import Phrases

# Add bigrams to docs (only ones that appear 15 times or more).
bigram = Phrases(docs, min_count=15)

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [21]:
# Create a dictionary representation of the documents, and filter out frequent and rare words.
from gensim.corpora import Dictionary
dictionary = Dictionary(docs)

# Remove rare and common tokens.
# Filter out words that occur too frequently or too rarely.
max_freq = 0.5
min_wordcount = 5
dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]
#MmCorpus.serialize("models/corpus.mm", corpus)

print('Number of unique tokens: %d' % len(dictionary))
print('Number of chunks: %d' % len(corpus))

Number of unique tokens: 5433
Number of chunks: 500


In [22]:
# models
from gensim.models import LdaMulticore
params = {'passes': 10, 'random_state': seed}
base_models = dict()
model = LdaMulticore(corpus=corpus, num_topics=4, id2word=dictionary, workers=6,
                passes=params['passes'], random_state=params['random_state'])

In [23]:
model.show_topics(num_words=5)

[(0,
  '0.003*"story" + 0.003*"face" + 0.002*"win" + 0.002*"hear" + 0.002*"game"'),
 (1,
  '0.003*"school" + 0.002*"use" + 0.002*"create" + 0.002*"issue" + 0.002*"ensure"'),
 (2,
  '0.003*"ensure" + 0.003*"understand" + 0.002*"risk" + 0.002*"story" + 0.002*"face"'),
 (3,
  '0.003*"school" + 0.003*"local" + 0.003*"believe" + 0.003*"protect" + 0.002*"issue"')]

In [24]:
model.show_topic(1,20)

[('school', 0.002589391),
 ('use', 0.002455004),
 ('create', 0.0023228377),
 ('issue', 0.0023038804),
 ('ensure', 0.0022757072),
 ('plan', 0.0020941573),
 ('win', 0.0020644234),
 ('let', 0.0020133485),
 ('situation', 0.002000675),
 ('forward', 0.0019986632),
 ('build', 0.001982574),
 ('future', 0.0019617334),
 ('person', 0.001954434),
 ('kid', 0.001937154),
 ('impact', 0.0019205007),
 ('face', 0.0018684922),
 ('leave', 0.0018411522),
 ('young', 0.0018350369),
 ('worker', 0.0018196257),
 ('student', 0.0018062431)]

In [25]:
sorted(model[corpus[0]],key=lambda x:x[1],reverse=True)

[(2, 0.4903521), (3, 0.48454005), (1, 0.016714452)]

In [36]:
# plot topics
import pyLDAvis.gensim_models

data =  pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
pyLDAvis.display(data)

In [27]:
# assignment
sent_to_cluster = list()
for n,doc in enumerate(corpus):
    if doc:
        cluster = max(model[doc],key=lambda x:x[1])
        sent_to_cluster.append(cluster[0])

In [28]:
# accuracy
from collections import Counter
for book, cluster in book_id.items():
    assignments = list()
    for real,given in zip(chunk_class,sent_to_cluster):
        if real == cluster:
            assignments.append(given)
    most_common,num_most_common = Counter(assignments).most_common(1)[0] # 4, 6 times
    print(book,":",most_common,"-",num_most_common)
    print("Accuracy:",num_most_common/limit)
    print("------")

women.txt : 1 - 181
Accuracy: 0.362
------
